In [ ]:
from pandas import DataFrame
import json
from lyricsgenius import Genius
from utils import ArtistSongs, Artists, Charts, Songs, Lyrics, DownloadChorus
import os

# Notebook para la extracción de los datos y creación del dataset principal

Este notebook  se centra en la extracción de datos de canciones utilizando la API de Genius. 

El objetivo principal es recopilar información relevante sobre canciones, incluyendo sus letras, artista, entre otros, y almacenarlas en un formato estructurado para su posterior análisis y uso con los modelos propuestos.

# Configuración de la API
Aquí se inicializa la conexión con la API de Genius utilizando el token proporcionado. Esto es esencial para autenticarse y acceder a los datos disponibles en Genius.


In [ ]:
genius = Genius('token')

In [ ]:
Artists.genius = genius
ArtistSongs.genius = genius
Charts.genius = genius
Songs.genius = genius
Lyrics.genius = genius

# Recolección de Datos
En esta sección, se descargan los datos de las listas de reproducción, canciones, letras y detalles de los artistas. Esto se hace utilizando las funciones proporcionadas por el módulo `utils`, que simplifican la interacción con la API de Genius.


In [ ]:
Charts.chart_download()
ArtistSongs.load_from_charts()
Lyrics.load_from_artist_songs()

# Mapeo de Información
Se crea una función para mapear la información de cada canción a un diccionario estándar. Esto incluye ID de la canción, nombre, artista, ID del artista, idioma de la canción y las letras. La función `map_song` es responsable de recopilar toda esta información en un formato estructurado.


In [ ]:
#Mapeo de Información: Se crea una función para mapear la información de cada canción a un diccionario estándar.
def id(song) -> int:
    return song._id

def name(song) -> str:
    return song.title

def artist(song) -> str:
    return song.artist

def artist_id(song) -> int:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('primary_artist', {}).get('id', None)

def language(song) -> str:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('language', None)

def lyrics(song) -> str:
    lyrics = Lyrics.lyrics(song._id)
    print(lyrics)
    lyrics = lyrics.split('\n')[1:]
    return "".join(lyrics)

def map_song(song) -> dict:
    return {
        'ID': id(song),
        'Name': name(song),
        'Artist': artist(song),
        'Artist ID': artist_id(song),
        'Language': language(song),
        'Lyrics': lyrics(song)
    }

songs = Songs.songs()
songs_dict = list(map(map_song, songs))

# Exportación de Datos
Finalmente, se guarda toda la información recopilada en un archivo CSV para su posterior análisis. Este paso es crucial para asegurar que los datos estén disponibles en un formato que sea fácil de importar y utilizar en otros proyectos o análisis de datos.


In [ ]:
#Exportación de Datos: Finalmente, se guarda toda la información recopilada en un archivo CSV para su posterior análisis.
directory = 'datasets'
if not os.path.exists(directory):
    os.makedirs(directory)
    
df = DataFrame(songs_dict)
df.to_csv('datasets/raw_songs.csv')

# Bonus - Extracción de estribillos

Como parte adicional creamos algunas funciones para extraer solo el estribillo de las canciones. Esto nos sirvio para construir nuestro datasert de validación que etiqueamos manualmente

In [ ]:
# Lista de artistas a extraer
artists = ["Kanye West", "Cardi B", "Lil Nas X", "Kelsea Ballerini", "Gabby Barrett", "Jason Aldean", "The Chainsmokers",
    "Marshmello", "Calvin Harris", "David Guetta", "Zedd", "Lorde", "Phoebe Bridgers",
    "Tame Impala", "Vampire Weekend", "Glass Animals", "Michael Bublé", "Josh Groban",
    "Rod Stewart", "Celine Dion", "Norah Jones"

In [ ]:
DownloadChorus.download_choruses(artists) #Posteriormente esta información la guardamos en un csv